In [1]:
import numpy as np
import pandas as pd
import os
eps = np.finfo(float).eps
from numpy import log2 as log

In [2]:
pwd

'C:\\Users\\mridu'

In [7]:
variable= pd.read_csv(r"D:\AI\Dataset.csv")
variable.head()

Outlook  Temp Humidity  Windy Play_Golf
0     Rainy   Hot     High  False        No
1     Rainy   Hot     High   True        No
2  Overcast   Hot     High  False       Yes
3     Sunny  Mild     High  False       Yes
4     Sunny  Cool   Normal  False       Yes

In [8]:
df = pd.DataFrame(variable,columns=['Outlook','Temp','Humidity','Windy','Play_Golf'])

In [9]:
df

Outlook  Temp Humidity  Windy Play_Golf
0      Rainy   Hot     High  False        No
1      Rainy   Hot     High   True        No
2   Overcast   Hot     High  False       Yes
3      Sunny  Mild     High  False       Yes
4      Sunny  Cool   Normal  False       Yes
5      Sunny  Cool   Normal   True        No
6   Overcast  Cool   Normal   True       Yes
7      Rainy  Mild     High  False        No
8      Rainy  Cool   Normal  False       Yes
9      Sunny  Mild   Normal  False       Yes
10     Rainy  Mild   Normal   True       Yes
11  Overcast  Mild     High   True       Yes
12  Overcast   Hot   Normal  False       Yes
13     Sunny  Mild     High   True        No

In [11]:

entropy_node = 0  #Initialize Entropy
values = df.Play_Golf.unique()  #Unique objects - 'Yes', 'No'
for value in values:
    fraction = df.Play_Golf.value_counts()[value]/len(df.Play_Golf)  
    entropy_node += -fraction*np.log2(fraction)

In [12]:
entropy_node

0.9402859586706311

In [15]:
attribute = 'Outlook'
target_variables = df.Play_Golf.unique()  #This gives all 'Yes' and 'No'
variables = df[attribute].unique()    #This gives different features in that attribute (like 'Sweet')
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df[attribute][df[attribute]==variable][df.Play_Golf ==target_variable]) #numerator
        den = len(df[attribute][df[attribute]==variable])  #denominator
        fraction = num/(den+eps)  #pi
        entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
    fraction2 = den/len(df)
    entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste

In [16]:
abs(entropy_attribute)

0.6935361388961914

In [19]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
  
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Play_Golf'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [20]:
tree=buildTree(df)
import pprint
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rainy': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
             'Sunny': {'Windy': {False: 'Yes', True: 'No'}}}}


In [21]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [22]:
inst=df.iloc[6]
inst

Outlook      Overcast
Temp             Cool
Humidity       Normal
Windy            True
Play_Golf         Yes
Name: 6, dtype: object

In [23]:
prediction=predict(inst,tree)
prediction


'Yes'

In [25]:
data={'Outlook': 'Overcast','Temp':'Cool','Humidity':'High','Windy':'False'}
inst=pd.Series(data)
prediction=predict(inst,tree)
prediction

'Yes'